# HyNet

In this notebook, we create a dataset of Armenian characters using the font Mk_Parz_U, and train a convolutional neural network to recognize characters from 56x56 images.

## Prepare

We build a dataset of Armenian characters based on the font Mk_Parz_U-Italic which resembles the one used in our target document :

![Document](test/samples/sample_denoised.png)

We use 56x56 pixel images and use filters and transformations to get a total of 34200 samples. We then split between train and test dataset with a ratio of 80/20.

In [37]:
import os
import pickle

from hynet.prepare import generate_classes, generate_dataset

N = 56  # 56x56 pixels
font_names = ["hynet/fonts/Mk_Parz_U-Italic"]
nb_classes = len(generate_classes())
train_dataset, test_dataset = generate_dataset(
    font_names=font_names, N=N, split_ratio=0.8
)

path = R"build/datasets/hynet"
os.makedirs(path, exist_ok=True)
with open(os.path.join(path, "train_dataset.pkl"), "wb") as f:
    pickle.dump(train_dataset, f)
with open(os.path.join(path, "test_dataset.pkl"), "wb") as f:
    pickle.dump(test_dataset, f)

2023-10-16 11:28:05,756 - root - INFO - Characters used     : ԱԲԳԴԵԶԷԸԹԺԻԼԽԾԿՀՁՂՃՄՅՆՇՈՉՊՋՌՍՎՏՐՑՒՓՔՕՖաբգդեզէըթժիլխծկհձղճմյնշոչպջռսվտրցւփքօֆ
2023-10-16 11:28:05,757 - root - INFO - Number of classes   : 76
2023-10-16 11:28:05,758 - root - INFO - Data augmentation   : 1 fonts, 5 rotations, 5 blur radiuses, 3 mode filters
2023-10-16 11:28:05,758 - root - INFO - Number of samples   : 5700


## Train

We train our LeNet-5 model to categorize the images into our 76 classes (alphabet with caps and small letters).

In [43]:
import os
import pickle

from hynet.model import LeNet, initialize_weights
from hynet.train import train
from torch.utils.data import DataLoader

batch_size = 16
nb_epochs = 100
learning_rate = 0.05
path = R"build/datasets/hynet"

train_dataset = pickle.load(open(os.path.join(path, "train_dataset.pkl"), "rb"))
test_dataset = pickle.load(open(os.path.join(path, "test_dataset.pkl"), "rb"))
train_dataloader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, drop_last=True
)
test_dataloader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, drop_last=True
)

# Initialize model
model_name = "LeNet-5"
model = LeNet(N, int(nb_classes))
model.apply(initialize_weights)

# Train
report = train(
    model_name=model_name,
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    nb_classes=int(nb_classes),
    batch_size=batch_size,
    nb_epochs=nb_epochs,
    learning_rate=learning_rate,
)

train_folder = "build/logs/train"
os.makedirs(train_folder, exist_ok=True)
report.save_model(os.path.join(train_folder, "model.pt"))  # save model
report.to_gif(os.path.join(train_folder, "report.gif"))  # plot report
report.to_csv(os.path.join(train_folder, "report.csv"))  # export full report

2023-10-16 11:30:59,106 - root - INFO - Epoch = 0 / 100, Training Loss = 0.27, Test Loss = 0.26, Test Accuracy = 27.55%
2023-10-16 11:31:01,288 - root - INFO - Epoch = 1 / 100, Training Loss = 0.25, Test Loss = 0.25, Test Accuracy = 52.11%
2023-10-16 11:31:03,417 - root - INFO - Epoch = 2 / 100, Training Loss = 0.24, Test Loss = 0.24, Test Accuracy = 63.56%
2023-10-16 11:31:05,578 - root - INFO - Epoch = 3 / 100, Training Loss = 0.23, Test Loss = 0.23, Test Accuracy = 74.47%
2023-10-16 11:31:08,047 - root - INFO - Epoch = 4 / 100, Training Loss = 0.23, Test Loss = 0.23, Test Accuracy = 77.46%
2023-10-16 11:31:10,699 - root - INFO - Epoch = 5 / 100, Training Loss = 0.23, Test Loss = 0.22, Test Accuracy = 80.72%
2023-10-16 11:31:13,306 - root - INFO - Epoch = 6 / 100, Training Loss = 0.22, Test Loss = 0.22, Test Accuracy = 83.63%
2023-10-16 11:31:15,872 - root - INFO - Epoch = 7 / 100, Training Loss = 0.22, Test Loss = 0.22, Test Accuracy = 84.60%
2023-10-16 11:31:18,415 - root - INFO - 

## Evaluation

We evaluate our model on the first batch and then show miscategorized samples.

In [44]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from hynet.prepare import generate_character_image, generate_classes
from PIL import Image, ImageFont

nb_batches = 10

df = report.dataframe
classes = generate_classes()

records = []
with torch.no_grad():
    for i, (inputs, labels) in enumerate(train_dataloader):
        # value = df[f'test_accuracy_{label}'].iloc[0]
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        for j, (T, label) in enumerate(zip(inputs, labels)):
            prediction = classes[predicted[j].item()]
            expected = classes[label.item()]
            if prediction != expected:
                records.append({"prediction": prediction, "expected": expected})
result = pd.DataFrame(records)

In [45]:
# Print most frequent miscategorized labels
result.apply(lambda x: f"{x['prediction']} => {x['expected']}", axis=1).value_counts()

տ => ա    62
զ => ղ    61
դ => գ    59
Ի => Ւ    59
ր => ը    59
Ժ => ժ    15
ծ => ժ    13
մ => ժ    10
ս => ժ     6
ձ => ժ     5
յ => ժ     5
լ => ժ     4
տ => ժ     2
Է => ժ     1
ց => ժ     1
Name: count, dtype: int64